In [40]:
import pandas as pd
from pykrx import stock
from pykrx import bond
import FinanceDataReader as fdr 

In [7]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = [
'https://spreadsheets.google.com/feeds',
'https://www.googleapis.com/auth/drive',
]

json_file_name = 'omega-tenure-341805-e2f94364a249.json'

credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

#삼성전자
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1Ws4AoSLvvlJs8ZOrN7lpF6Y1PWwrZUVssF71Awatmuw/edit#gid=448337019'

# 스프레스시트 문서 가져오기 
doc = gc.open_by_url(spreadsheet_url)

# 시트 선택하기
worksheet = doc.worksheet('df')

삼성전자

In [21]:
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1IBclG_tUnFOKIZE_Y46_4TkOzVmsFWf67DJyZVbS0Us/edit#gid=0'

# 스프레스시트 문서 가져오기 
doc = gc.open_by_url(spreadsheet_url)

# 시트 선택하기
worksheet_fund = doc.worksheet('시트1')

In [22]:
df_fund = stock.get_market_fundamental("20200102", "20220529", "005930")

In [23]:
df_fund['Date'] = df_fund.index.values
df_fund['Date'] = pd.to_datetime(df_fund['Date'], format="%m/%d/%Y")
df_fund['Date'] = df_fund['Date'].astype(str)
df_fund['Date']
worksheet_fund.update([df_fund.columns.values.tolist()] + df_fund.values.tolist())

{'spreadsheetId': '1IBclG_tUnFOKIZE_Y46_4TkOzVmsFWf67DJyZVbS0Us',
 'updatedRange': "'시트1'!A1:G596",
 'updatedRows': 596,
 'updatedColumns': 7,
 'updatedCells': 4172}

In [24]:
#LG 데이터와 SK 데이터 불러오기
LG_fund = stock.get_market_fundamental("20200102", "20220529", "066570")
SK_fund = stock.get_market_fundamental("20200102", "20220529", "000660")

LG 데이터 덮어씌우기

In [25]:
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1m32D0JHJ_dkHwdHj1jrkVqTiFljezLzVejhZQkvwGRA/edit#gid=0'

# 스프레스시트 문서 가져오기 
doc = gc.open_by_url(spreadsheet_url)

# 시트 선택하기
worksheet_fund = doc.worksheet('시트1')

In [26]:
LG_fund['Date'] = LG_fund.index.values

In [27]:
LG_fund['Date'] = pd.to_datetime(LG_fund['Date'], format="%m/%d/%Y")

In [28]:
LG_fund['Date'] = LG_fund['Date'].astype(str)

In [29]:
worksheet_fund.update([LG_fund.columns.values.tolist()] + LG_fund.values.tolist())

{'spreadsheetId': '1m32D0JHJ_dkHwdHj1jrkVqTiFljezLzVejhZQkvwGRA',
 'updatedRange': "'시트1'!A1:G596",
 'updatedRows': 596,
 'updatedColumns': 7,
 'updatedCells': 4172}

감정 크롤링

In [69]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup
import re
from konlpy.tag import Okt
import random

In [77]:
df = fdr.DataReader(symbol='005930', start="20200102") # 삼성전자, 2020년~현재
df.columns = ['시가', '상한가', '하한가', '종가', '거래량', '등락률']

In [87]:
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1Ws4AoSLvvlJs8ZOrN7lpF6Y1PWwrZUVssF71Awatmuw/edit#gid=448337019'

# 스프레스시트 문서 가져오기 
doc = gc.open_by_url(spreadsheet_url)

worksheet = doc.worksheet('df')
df['Date'] = df.index.values
df['Date'] = df['Date'].astype(str)

In [79]:
low = ["실패", "급락", "하락", "조롱", "비하", "공포", "멈춰", "애증", "폭락", "매수세", "중단", "거북이걸음", "샤오미", "사과", "조정", "요구", "부진하다", "이동", "포기", "코로나", "깨다", "우려", "부실", "불황", "적자", "불만", "철수", "전쟁", "낭패", "부진하다", "장주", "구속", "터지다", "수감", "구조", "굴욕", "그치다", "과징금", "주춤", "폐렴", "예판", "디스플레이", "자사주"]
up = ["껑충", "반등", "대장", "추천주", "추격", "출시", "신제품", "양산", "집중", "차지", "공개", "성공", "투자", "가즈아", "최고", "격차", "간다", "다시", "에이스", "최고", "혁신", "흑자", "수출", "추진", "부족하다", "잇따르다", "생산", "공개", "가속", "적극", "대응", "나가다", "행진", "장밋빛", "탈락", "압박", "침몰"]

In [64]:
date_index = pd.DataFrame()

In [65]:
date_index['Date'] = pd.to_datetime(df.index)
years = date_index['Date'].dt.year
months = date_index['Date'].dt.month
days = date_index['Date'].dt.day

In [70]:
okt = Okt()

In [71]:
import time
result_contents = []
driver = webdriver.Chrome(ChromeDriverManager().install())
for depth in range(0,len(years)):
    rand_value = random.randrange(1,3)
    time.sleep(rand_value)
    url = f"https://www.google.com/search?q=삼성전자&biw=766&bih=741&sxsrf=APq-WBudo9b7nlfEOlDbyptJCTVYpqrq8A%3A1645011742667&source=lnt&tbs=cdr%3A1%2Ccd_min%3A{months[depth]}%2F{days[depth]}%2F{years[depth]}%2Ccd_max%3A{months[depth]}%2F{days[depth]}%2F{years[depth]}&tbm=nws"
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html)
    v_title = soup.select(".mCBkyc.y355M.JQe2Ld.nDgy9d")
    for i in range(len(v_title)):
        v_title[i] = v_title[i].text
        v_title[i] = re.sub('[^0-9a-zA-Zㄱ-힗 ]', '', v_title[i])
        v_title[i] = Okt().pos(v_title[i], norm=True, stem=True)
        v_title[i] = [x for x, y in v_title[i] if y in ['Noun', 'Adjective', 'Verb']]
        point = 0
        percentile1 = len(v_title[i]) * (2/10)
        percentile2 = len(v_title[i]) * (4/10)
        percentile3 = len(v_title[i]) * (6/10)
        percentile4 = len(v_title[i]) * (8/10)
        for length in range(len(v_title[i])):
            if v_title[i][length] in low:
                if length <= percentile1:
                    point -= 1
                elif length < percentile2 and length > percentile1:
                    point -= 3
                elif length < percentile3 and length >= percentile2:
                    point -= 5
                elif length >= percentile3:
                    point -= 10

            if v_title[i][length] in up:
                if length <= percentile1:
                    point += 1
                elif length < percentile2 and length > percentile1:
                    point += 3
                elif length < percentile3 and length >= percentile2:
                    point += 5
                elif length >= percentile3:
                    point += 10
        try:
            v_title[i] = point / len(v_title[i])
            sum_title = sum(v_title)
        except:
            sum_title = 0


    v_content = soup.select(".GI74Re.nDgy9d")

    for i in range(len(v_content)):
        v_content[i] = v_content[i].text
        v_content[i] = re.sub('[^0-9a-zA-Zㄱ-힗 ]', '', v_content[i])
        v_content[i] = Okt().pos(v_content[i], norm=True, stem=True)
        v_content[i] = [x for x, y in v_content[i] if y in ['Noun', 'Adjective', 'Verb']]

        point = 0
        percentile1 = len(v_content[i]) * (2/10)
        percentile2 = len(v_content[i]) * (4/10)
        percentile3 = len(v_content[i]) * (6/10)
        percentile4 = len(v_content[i]) * (8/10)

        for length in range(len(v_content[i])):
            if v_content[i][length] in low:
                if length <= percentile1:
                    point -= 1
                elif length < percentile2 and length > percentile1:
                    point -= 3
                elif length < percentile3 and length >= percentile2:
                    point -= 5
                elif length >= percentile3:
                    point -= 10

            if v_content[i][length] in up:
                if length <= percentile1:
                    point += 1
                elif length < percentile2 and length > percentile1:
                    point += 3
                elif length < percentile3 and length >= percentile2:
                    point += 5
                elif length >= percentile3:
                    point += 10
        try:
            v_content[i] = point / len(v_content[i])
            sum_content = sum(v_content)
        except:
            sum_content = 0
    

    result = sum_title + sum_content

    result_contents.append(result)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\byeongsu\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [85]:
df['감정'] = result_contents

In [88]:
# 업데이트
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1Ws4AoSLvvlJs8ZOrN7lpF6Y1PWwrZUVssF71Awatmuw',
 'updatedRange': 'df!A1:H596',
 'updatedRows': 596,
 'updatedColumns': 8,
 'updatedCells': 4768}

삼성전자 및 LG SK하이닉스 PER PBR ROE

In [72]:
#삼성전자와 친구들
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1N5Gv5H5CgYYPzgT8Jrl4KeIdmrWtXAqqcBsVQ-tcS-M/edit#gid=0'

# 스프레스시트 문서 가져오기 
doc = gc.open_by_url(spreadsheet_url)

# 시트 선택하기
worksheet_fund = doc.worksheet('시트1')

In [73]:
samsung_data = pd.DataFrame()

In [74]:
samsung_data['삼성 PBR'] = df_fund['PBR']
samsung_data['LG전자 PBR'] = LG_fund['PBR']
samsung_data['SK하이닉스 PBR'] = SK_fund['PBR']
samsung_data['삼성 PER'] = df_fund['PER']
samsung_data['LG전자 PER'] = LG_fund['PER']
samsung_data['SK하이닉스 PER'] = SK_fund['PER']

In [75]:
samsung_data['삼성 ROE'] = df_fund['PBR'] / df_fund['PER']

In [80]:
df['Date'] = df['Date'].astype(str)

In [81]:
samsung_data['Date'] = LG_fund['Date']

In [82]:
worksheet_fund.update([samsung_data.columns.values.tolist()] + samsung_data.values.tolist())

{'spreadsheetId': '1N5Gv5H5CgYYPzgT8Jrl4KeIdmrWtXAqqcBsVQ-tcS-M',
 'updatedRange': "'시트1'!A1:H596",
 'updatedRows': 596,
 'updatedColumns': 8,
 'updatedCells': 4768}